In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np
from torchvision.models import resnet18

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

torch.manual_seed(42)
np.random.seed(42)

class_names = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    ]
)

train_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform
)

train_size = len(train_dataset)
indices = list(range(train_size))
np.random.shuffle(indices)

split = int(np.floor(0.2 * train_size))
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

batch_size = 128

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2
)

val_loader = DataLoader(
    train_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=2
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=2
)

from torchvision.models import ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=5
)


def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels


num_epochs = 100
best_val_loss = float('inf')
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(
        model, train_loader, optimizer, criterion, device
    )
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_10class.pth')

    print(
        f'Epoch {epoch + 1}/{num_epochs} | '
        f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | '
        f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}'
    )

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

model.load_state_dict(torch.load('best_model_10class.pth'))

test_loss, test_acc, test_preds, test_labels = evaluate(
    model, test_loader, criterion, device
)
print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}')

In [ ]:
def binary_accuracy_from_multiclass(model, data_loader, device):
    model.eval()
    binary_correct = 0
    binary_total = 0

    # treat the following classes as positive Shirt(6), Dress(3), Coat(4), Pullover(2), T-shirt/top(0)
    positive_classes = [0, 2, 3, 4, 6]

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            class_scores = torch.softmax(outputs, dim=1)

            binary_true_labels = torch.tensor(
                [
                    1 if label.item() in positive_classes else 0
                    for label in labels
                ],
                device=device,
            )

            positive_scores = torch.zeros(class_scores.shape[0], device=device)
            for cls in positive_classes:
                positive_scores += class_scores[:, cls]

            negative_scores = torch.zeros(class_scores.shape[0], device=device)
            for cls in range(10):
                if cls not in positive_classes:
                    negative_scores += class_scores[:, cls]

            binary_preds = torch.where(
                positive_scores >= negative_scores,
                torch.ones_like(positive_scores),
                torch.zeros_like(positive_scores),
            )

            binary_correct += (binary_preds == binary_true_labels).sum().item()
            binary_total += labels.size(0)

    binary_accuracy = binary_correct / binary_total if binary_total > 0 else 0

    return binary_accuracy


binary_acc = binary_accuracy_from_multiclass(model, test_loader, device)